In [1]:
import requests
import json
import pandas as pd

In [2]:
csv_path = "show list.csv"

shows = pd.read_csv(csv_path)
shows = shows["Title"]
shows.head(20)

0                The Amazing Spider-Man
1                   The Incredible Hulk
2                            Spider-Man
3                                 Blade
4                Agents of S.H.I.E.L.D.
5                          Agent Carter
6                             Daredevil
7                         Jessica Jones
8                             Luke Cage
9                                Legion
10                            Iron Fist
11                        The Defenders
12                             Inhumans
13                           The Gifted
14                         The Punisher
15                             Runaways
16                       Cloak & Dagger
17                             Helstrom
18                          WandaVision
19    The Falcon and the Winter Soldier
Name: Title, dtype: object

In [3]:
base_url = "http://api.tvmaze.com/search/shows?q="

In [4]:
response = requests.get(base_url).json()
print(json.dumps(response, indent=4, sort_keys=True))

[]


In [5]:

responses = []
show_info = []
show_name = []
show_id = []
tvrage = []
thetvdb = []
imdb = []



for show in shows: 
    mod_url = base_url + show
    
#     print(json.dumps(response_json, indent=4, sort_keys=True))

    
    try:
        show_data = requests.get(mod_url)
        response_json = show_data.json()
        show_name.append(response_json[0]['show']['name'])
        show_id.append(response_json[0]['show']['id'])
        tvrage.append(response_json[0]['show']['externals']['tvrage'])
        thetvdb.append(response_json[0]['show']['externals']['thetvdb'])
        imdb.append(response_json[0]['show']['externals']['imdb'])                       
        
    except:
        pass
                         
                         
                 

In [6]:
show_data_df = pd.DataFrame({"Show Name": show_name, "Show ID": show_id, "tvrage": tvrage, "the tvdb": thetvdb, "imdb": imdb})
show_data_df.head(100)

,Show Name,Show ID,tvrage,the tvdb,imdb
0,The Amazing Spider-Man,7623,NaN,78453.0,tt0076975
1,The Incredible Hulk,2151,5870.0,71746.0,tt0077031
2,Spider-Man,1611,5294.0,73750.0,tt0112175
3,Blade,8331,30524.0,249846.0,tt1988235
4,Marvel's Agents of S.H.I.E.L.D.,31,32656.0,263365.0,tt2364582
...,...,...,...,...,...
79,Marvel's Ant-Man Shorts,29100,NaN,329035.0,None
80,Marvel Rising: Initiation,37823,NaN,350537.0,tt9073722
81,Baymax!,52349,NaN,NaN,None
82,Baymax!,52349,NaN,NaN,None


In [7]:
show_data_df['Show ID'] = show_data_df['Show ID'].astype(str)

In [14]:
cast_response = []
character_name = []
character_id = []


for ids in show_id: 
    cast_url = (f"http://api.tvmaze.com/shows/{ids}?embed[]=episodes&embed[]=cast")
    
    
    try:
        cast_data = requests.get(cast_url).json()
        cast_response.append(cast_data)
        for i in range(0, 150): 
                character_name.append(cast_data['_embedded']['cast'][i]['character']['name'])
                character_id.append(cast_data['_embedded']['cast'][i]['character']['id'])

    except:
        pass
                         
                   

In [17]:
character_data_df = pd.DataFrame({"Character Name": character_name, "character ID": character_id})
character_data_df.head()

,Character Name,character ID
0,Peter Parker/Spider-Man,196352
1,Captain Barbera,365202
2,J. Jonah Jameson,365203
3,J. Jonah Jameson,365203
4,Rita Conway,365204


In [10]:
episode_id = []
episode_season = []
episode_num = []
episode_air = []
episode_name = []

for ids in show_id: 
    cast_url = (f"http://api.tvmaze.com/shows/{ids}?embed[]=episodes&embed[]=cast")
    
    
    try:
        cast_data = requests.get(cast_url).json()
        for i in range(0, 150): 
                episode_id.append(cast_data['_embedded']['episodes'][i]['id'])
                episode_season.append(cast_data['_embedded']['episodes'][i]['season'])
                episode_num.append(cast_data['_embedded']['episodes'][i]['number'])
                episode_air.append(cast_data['_embedded']['episodes'][i]['airstamp'])
                episode_name.append(cast_data['_embedded']['episodes'][i]['name'])
    except:
        pass
                         
                   

In [20]:
episode_data_df = pd.DataFrame({"Episode Name": episode_name, "Episode ID": episode_id, "Episode Name": episode_name, "Episode Season": episode_season, "Episode Number": episode_num, "Episode Air Date": episode_air})
episode_data_df.head()

,Episode Name,Episode ID,Episode Season,Episode Number,Episode Air Date
0,The Amazing Spider-Man,429649,1,1,1977-09-15T00:00:00+00:00
1,The Deadly Dust: Part 1,429650,1,2,1978-04-06T01:00:00+00:00
2,The Deadly Dust: Part 2,429651,1,3,1978-04-13T01:00:00+00:00
3,The Curse of Rava,429652,1,4,1978-04-20T01:00:00+00:00
4,Night of the Clones,429653,1,5,1978-04-27T01:00:00+00:00
